In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.interpolate import griddata

In [12]:
# the expected uniform mesh (out)
lon_uni = np.arange(-117.0,-108.8,0.1)
lat_uni = np.arange(60.7,62.9,0.1)
lons,lats = np.meshgrid(lon_uni, lat_uni)
grid_out = np.concatenate([lons.reshape(-1,1), lats.reshape(-1,1)], axis=1)

In [13]:
np.shape(grid_out)

(1826, 2)

In [14]:
# ds = xr.open_dataset(r'./out.nc')
era5_uu = xr.open_dataset(r'./U_2000-9-29_10-3.nc')
era5_vv = xr.open_dataset(r'./V_2000-9-29_10-3.nc')
era5_p = xr.open_dataset(r'./p_2000-9-29_10-3.nc')

lon = era5_p.lon.values
lat = era5_p.lat.values
# lons,lats = np.meshgrid(lon, lat)
# grid_in = np.concatenate([lons.reshape(-1,1), lats.reshape(-1,1)], axis=1)
grid_in = np.concatenate([lon.reshape(-1,1), lat.reshape(-1,1)], axis=1)

In [17]:
import os
filename = 'atmos_2000-9-29_10-03.nc'
os.system("rm ./"+filename)
save_file = r'./'+filename

date_ran = pd.date_range("2000-09-29 00", "2000-10-03 00", freq="1h")
time_count = len(date_ran)

# grid_out = np.vstack((points['lon'], points['lat'])).T

u10_out = np.empty((time_count,len(lat_uni), len(lon_uni)))
v10_out = np.empty((time_count,len(lat_uni), len(lon_uni)))
sp_out = np.empty((time_count,len(lat_uni), len(lon_uni)))

# interpolation
# outputs =  np.empty((0,3))
for itime in range(time_count):
    u10_in = era5_uu.UU.sel(time=date_ran[itime], method='nearest')
    v10_in = era5_vv.VV.sel(time=date_ran[itime], method='nearest')
    sp_in = era5_p.slp.sel(time=date_ran[itime], method='nearest')
    u10_out_interp = griddata(grid_in, np.array(u10_in).flatten(), grid_out, method='linear')
    v10_out_interp = griddata(grid_in, np.array(v10_in).flatten(), grid_out, method='linear')
    sp_out_interp = griddata(grid_in, np.array(sp_in).flatten(), grid_out, method='linear')

    # print(str(np.size(u10_out_interp)))

    u10_out[itime] = u10_out_interp.reshape(len(lat_uni), len(lon_uni))
    v10_out[itime] = v10_out_interp.reshape(len(lat_uni), len(lon_uni))
    sp_out[itime] = sp_out_interp.reshape(len(lat_uni), len(lon_uni))
#     output = np.vstack((points['id'],u10_out,v10_out,sp_out)).T
#     outputs = np.concatenate([outputs, output])
    print(str(itime)+': '+str(date_ran[itime]))

rm: cannot remove './atmos_2000-9-29_10-03.nc': No such file or directory


0: 2000-09-29 00:00:00
1: 2000-09-29 01:00:00
2: 2000-09-29 02:00:00
3: 2000-09-29 03:00:00
4: 2000-09-29 04:00:00
5: 2000-09-29 05:00:00
6: 2000-09-29 06:00:00
7: 2000-09-29 07:00:00
8: 2000-09-29 08:00:00
9: 2000-09-29 09:00:00
10: 2000-09-29 10:00:00
11: 2000-09-29 11:00:00
12: 2000-09-29 12:00:00
13: 2000-09-29 13:00:00
14: 2000-09-29 14:00:00
15: 2000-09-29 15:00:00
16: 2000-09-29 16:00:00
17: 2000-09-29 17:00:00
18: 2000-09-29 18:00:00
19: 2000-09-29 19:00:00
20: 2000-09-29 20:00:00
21: 2000-09-29 21:00:00
22: 2000-09-29 22:00:00
23: 2000-09-29 23:00:00
24: 2000-09-30 00:00:00
25: 2000-09-30 01:00:00
26: 2000-09-30 02:00:00
27: 2000-09-30 03:00:00
28: 2000-09-30 04:00:00
29: 2000-09-30 05:00:00
30: 2000-09-30 06:00:00
31: 2000-09-30 07:00:00
32: 2000-09-30 08:00:00
33: 2000-09-30 09:00:00
34: 2000-09-30 10:00:00
35: 2000-09-30 11:00:00
36: 2000-09-30 12:00:00
37: 2000-09-30 13:00:00
38: 2000-09-30 14:00:00
39: 2000-09-30 15:00:00
40: 2000-09-30 16:00:00
41: 2000-09-30 17:00:00
42

In [18]:
print(u10_out.shape)
print(grid_out.shape)

(97, 22, 83)
(1826, 2)


In [23]:
# create NetCDF file
ds1 = xr.Dataset(
    data_vars={
        "u10": (("time", "lat", "lon"), np.float32(u10_out)),
        "v10": (("time", "lat", "lon"), np.float32(v10_out)),
        "sp": (("time", "lat", "lon"), np.float32(sp_out)),
    },
    coords={
        # "node": np.arange(1, npoint+1),
        "time": np.arange(1, time_count+1),
        "lat": np.arange(1, len(lat_uni)+1),
        "lon": np.arange(1, len(lon_uni)+1),
    },
)

In [24]:
print ('saving to ', save_file)
ds1.to_netcdf(path=save_file)

saving to  ./atmos_2000-9-29_10-03.nc
